In [1]:
import requests
import pandas as pd
from datetime import datetime
import git

path = "/home/user/commits/commit_messages_generation/"

In [6]:
# df = pd.DataFrame(columns=['repository_ID', 'name', 'URL', 'created_date', 'commits_api_link', 'number_of_stars'])

for page in range(1, 11):
    results = requests.get(
       f'https://api.github.com/search/repositories?q=language:python&sort=stars&order=desc&per_page=100&page={page}').json()

    for repo in results['items']:
        d_tmp = {'repository_ID': repo['id'],
                 'name': repo['name'],
                 'URL': repo['html_url'],
                 'created_date': datetime.strptime(repo['created_at'], '%Y-%m-%dT%H:%M:%SZ'),
                 'commits_api_link': repo['commits_url'].split('{')[0],
                 'number_of_stars': repo['stargazers_count']}

        df = pd.concat([df, pd.DataFrame(d_tmp, index=[0])], ignore_index=True)

df.head()

,repository_ID,name,URL,created_date,commits_api_link,number_of_stars
0,2678181,Vintage,https://github.com/sublimehq/Vintage,2011-10-31 01:59:44,https://api.github.com/repos/sublimehq/Vintage...,569
1,226997592,pvcnn,https://github.com/mit-han-lab/pvcnn,2019-12-10 00:52:20,https://api.github.com/repos/mit-han-lab/pvcnn...,564
2,45493643,nzbhydra,https://github.com/theotherp/nzbhydra,2015-11-03 20:30:38,https://api.github.com/repos/theotherp/nzbhydr...,560
3,3518605,feedhq,https://github.com/feedhq/feedhq,2012-02-22 20:28:09,https://api.github.com/repos/feedhq/feedhq/com...,559
4,87509364,learnPython,https://github.com/rieuse/learnPython,2017-04-07 05:48:17,https://api.github.com/repos/rieuse/learnPytho...,553


In [10]:
df.head()

,repository_ID,name,URL,created_date,commits_api_link,number_of_stars
0,2678181,Vintage,https://github.com/sublimehq/Vintage,2011-10-31 01:59:44,https://api.github.com/repos/sublimehq/Vintage...,569
1,226997592,pvcnn,https://github.com/mit-han-lab/pvcnn,2019-12-10 00:52:20,https://api.github.com/repos/mit-han-lab/pvcnn...,564
2,45493643,nzbhydra,https://github.com/theotherp/nzbhydra,2015-11-03 20:30:38,https://api.github.com/repos/theotherp/nzbhydr...,560
3,3518605,feedhq,https://github.com/feedhq/feedhq,2012-02-22 20:28:09,https://api.github.com/repos/feedhq/feedhq/com...,559
4,87509364,learnPython,https://github.com/rieuse/learnPython,2017-04-07 05:48:17,https://api.github.com/repos/rieuse/learnPytho...,553


In [2]:
def get_repo_commits(repository):
    def check_message(x):
        if x.isascii() is False:
            return False
        return True

    commits_df = pd.DataFrame(columns=['repo_name', 'commit_message', 'files_changed', 'commit_changes', "hash"])
    commits = repository.iter_commits()
    repo_name = repository.working_dir.split('/')[-1]
    for commit in commits:
        message = commit.message
        if check_message(message) is False:
            continue
        if 0 > len(message.strip().split()) > 512:
            continue

        commit_sha1 = commit.hexsha
        commit = repository.commit(commit_sha1)
        try:
            commit_before = repository.commit(commit_sha1 + '~1')
        except Exception:
            continue

        diff_text = repository.git.diff(commit_before, commit, unified=False)
        diff_text_divided = diff_text.split('diff --git')
        diff_text_divided = diff_text_divided[1:]
        if len(diff_text_divided) > 100: #skip commits with too many files changed
            continue
        for diffs in diff_text_divided:
            if diffs != '':
                try:
                    filename = diffs.split(' b/')[0].split(' a/')[1]
                except IndexError:
                    try:
                        filename = diffs.split('b/')[0].split('a/')[1]  #for chines file names
                    except:
                        filename = 'unknown'  #TODO: should drop this after parsing all repos
                patch = diffs
                commits_df = pd.concat([commits_df, pd.DataFrame({'repo_name': repo_name,
                                                                  'commit_message': message,
                                                                  'files_changed': filename,
                                                                  'commit_changes': patch,
                                                                  'hash': commit_sha1}, index=[0])],
                                       ignore_index=True)
    return commits_df

In [3]:
import os
repos = []


def download_repos(df):
    # for index, row in df.iterrows():
    #     print('Downloading repository: ' + row['name'])

    #     if row["name"] == "api":
    #         continue

    #     try:
    #         repo = git.Repo.clone_from(row['URL'], '../data/raw/repositories/' + row['name'])
    #     # if repo is already downloaded
    #     except git.exc.GitCommandError:
    #         repo = git.Repo('../data/raw/repositories/' + row['name'])
    #     repos.append(repo)
    #     print('Repository downloaded: ' + row['name'])

    # get only downloaded repos
    folder = path + '/data/raw/repositories/'
    sub_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]
    for el in sub_folders:
        repo = git.Repo(path + '/data/raw/repositories/' + el)
        repos.append(repo)
        print("Repository dowloaded: " + el)

In [ ]:
# download_repos(df)
download_repos("123")

In [5]:
len(repos)

1242

In [6]:
total_commits_df = pd.read_pickle(f'{path}/data/raw/commits_new_v3.pickle')
parsed = total_commits_df.repo_name.unique()
repos = [x for x in repos if x.working_dir.split('/')[-1] not in parsed]

total_commits_df = pd.read_pickle(f'{path}/data/raw/commits_new_v4.pickle')
parsed = total_commits_df.repo_name.unique()
repos = [x for x in repos if x.working_dir.split('/')[-1] not in parsed]

total_commits_df.shape

(215, 5)

In [7]:
import gc
gc.collect()
print(len(repos))

928


In [12]:
import time
# total_commits_df = pd.DataFrame(columns=['repo_name', 'commit_message', 'files_changed', 'commit_changes', "hash"])
# limit = 2500000

for idx, repo in enumerate(repos):
    # if total_commits_df.shape[0] > limit:
    #     break
    
    #check if repos with this name is already in the dataframe
    
    if len(total_commits_df[total_commits_df['repo_name'] == repo.working_dir.split('/')[-1]]) != 0:
        print('Skipping: ' + repo.working_dir.split('/')[-1] + ' as it is already parsed')
        continue
    start_time = time.time()
    commits_number = len(list(repo.iter_commits()))
    if repo.working_dir.split('/')[-1] == 'AutoEq':
        continue # some bug with this repo
    if commits_number > 3000:
        print('Skipping: ' + repo.working_dir.split('/')[-1] + 'as it have to much commits ' +  str(commits_number))
        continue
    print('Parsing: ' + repo.working_dir.split('/')[-1] + ' with ' + str(commits_number) + ' commits')
    total_commits_df = pd.concat([total_commits_df, get_repo_commits(repo)], ignore_index=True)
    print('Parsed in ' + str(time.time() - start_time) + ' seconds')
    total_commits_df.to_pickle(f'{path}/data/raw/commits_new_v4.pickle') # save after each repo

Skipping: skllas it have to much commits 3693
Skipping: the-monitor as it is already parsed
Skipping: pipenvas it have to much commits 8355
Skipping: research-charnet as it is already parsed
Skipping: python-pattern-matching as it is already parsed
Skipping: ReCirq as it is already parsed
Skipping: onnx-caffe2 as it is already parsed
Skipping: StyleGAN-Tensorflow as it is already parsed
Skipping: django-startproject as it is already parsed
Skipping: poutyne as it is already parsed
Skipping: ask-multiple-pdfs as it is already parsed
Skipping: lushi_script as it is already parsed
Skipping: nematus as it is already parsed
Skipping: django-rest-framework-mongoengine as it is already parsed
Skipping: MachineLearning as it is already parsed
Skipping: shrapnel as it is already parsed
Skipping: docker-locust as it is already parsed
Skipping: Text-Pairs-Relation-Classification as it is already parsed
Skipping: EXCELntDonut as it is already parsed
Skipping: blueoil as it is already parsed
Skippi

In [12]:
total_commits_df.shape

(1174385, 5)

In [11]:
total_commits_df.head()

,repo_name,commit_message,files_changed,commit_changes,hash
0,pandora,展示模型名追加tags (#951)\n\n原来格式化的结果可能存在同名model的情况，让...,src/pandora/bots/legacy.py,a/src/pandora/bots/legacy.py b/src/pandora/bo...,8f2d776f6a72c9d5973a24ecbea4b5ace8c7338d
1,pandora,Add the missing Docker env: PANDORA_THREADS\n\...,bin/startup.sh,a/bin/startup.sh b/bin/startup.sh\nindex c6eb...,f33d09288498be4921e244d674845170967053b9
2,pandora,Add the missing Docker env: PANDORA_THREADS\n\...,doc/wiki.md,a/doc/wiki.md b/doc/wiki.md\nindex 20330da..6...,f33d09288498be4921e244d674845170967053b9
3,pandora,Add the missing Docker env: PANDORA_THREADS\n\...,doc/wiki_en.md,a/doc/wiki_en.md b/doc/wiki_en.md\nindex 2f93...,f33d09288498be4921e244d674845170967053b9
4,pandora,Add the missing Docker env: PANDORA_THREADS\n\...,src/pandora/__init__.py,a/src/pandora/__init__.py b/src/pandora/__ini...,f33d09288498be4921e244d674845170967053b9


In [30]:
import numpy as np

idx = total_commits_df["hash"].value_counts().index[total_commits_df["hash"].value_counts() == 1]
temp_df = total_commits_df[total_commits_df["hash"].isin(idx)]


print("Number of commits with one changed file: ", end="")
print(temp_df[temp_df["files_changed"].str.contains(".py")].shape[0])

Number of commits with one changed file: 92124
